In [1]:
from qiskit import QuantumCircuit
from qiskit.transpiler import CouplingMap
import random
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes.routing.sabre_swap_v0_20_      import SabreSwap 
from qiskit.transpiler.passes import ApplyLayout, FullAncillaAllocation, \
                                     EnlargeWithAncilla
from qiskit.transpiler.passes.layout.sabre_layout    import SabreLayout
from qiskit.qasm2 import dump 
from zou_circuits import create_parallel_circuit, apply_swaps_and_get_matching_circuit
random.seed(42)

In [3]:
num_qubits = 27
num_layers = 10
cm = CouplingMap.from_ring(num_qubits)


qc = create_parallel_circuit(num_qubits, num_layers, cm)

# Get the matching circuit after swaps are applied
matching_circuit = apply_swaps_and_get_matching_circuit(qc)

# Get transpiled circuit of the matching circuit
rp = SabreSwap(cm, seed=42)
lp = SabreLayout(cm, routing_pass=rp)

pm = PassManager([
    lp,
    FullAncillaAllocation(cm),
    EnlargeWithAncilla(),
    ApplyLayout(),
    rp
])

qc_transpiled = pm.run(matching_circuit)
qc_transpiled.draw()

# Print the original and modified circuits
print("Original Circuit:")
print(qc)
print("\nModified Circuit after applying SWAPs:")
print(matching_circuit.draw())
print("\nTranspiled Circuit of the modified circuit:")
print(qc_transpiled.draw())

Original Circuit:
                                    ┌───┐     ┌───┐        ┌───┐             »
 q_0: ───────■────X────X────────────┤ X ├──X──┤ X ├──────X─┤ X ├──────X───■──»
      ┌───┐  │    │    │            └─┬─┘  │  └─┬─┘      │ └─┬─┘      │   │  »
 q_1: ┤ X ├──┼────X────X─────────■────┼────X────■────X───┼───■────X───┼───┼──»
      └─┬─┘  │                 ┌─┴─┐  │              │   │        │   │   │  »
 q_2: ──■────┼──────────────X──┤ X ├──┼────X─────────X───┼────────X───┼───┼──»
             │              │  ├───┤  │    │  ┌───┐      │            │   │  »
 q_3: ───────┼─────────■────X──┤ X ├──┼────X──┤ X ├──────┼───X────────┼───┼──»
      ┌───┐  │       ┌─┴─┐     └─┬─┘  │       └─┬─┘      │   │  ┌───┐ │   │  »
 q_4: ┤ X ├──┼────X──┤ X ├──X────■────┼─────────■────X───┼───X──┤ X ├─┼───┼──»
      └─┬─┘  │    │  ├───┤  │         │       ┌───┐  │   │      └─┬─┘ │   │  »
 q_5: ──■────┼────X──┤ X ├──X────■────┼────X──┤ X ├──X───┼───■────■───┼───┼──»
             │       └─┬─┘     ┌─┴

In [4]:
print("\nOriginal Circuit Depth:", qc.depth())
print("Decomposed Original Circuit Depth:", qc.decompose().depth())

print("\nTranspiled Circuit Depth:", qc_transpiled.depth())
print("Decomposed Transpiled Circuit Depth:", qc_transpiled.decompose().depth())

print("\nModified Circuit Depth:", matching_circuit.depth())



Original Circuit Depth: 20
Decomposed Original Circuit Depth: 40

Transpiled Circuit Depth: 31
Decomposed Transpiled Circuit Depth: 74

Modified Circuit Depth: 10


In [5]:
def create_parallel_circuits_set(num_qubits, num_layers_list, coupling_map):
    # Create a set of quantum circuits with the given number of qubits and number of layers
    qc_list = [] # a list of tuples (original_circuit, matching_circuit)
    for num_layers in num_layers_list:
        qc_orig = create_parallel_circuit(num_qubits, num_layers, coupling_map)
        file_answer = f"experiment_setup/circuits/zou_circuits/answers_ring_27/parallel_circuit_{num_qubits}_{num_layers}.qasm"
        dump(qc_orig, file_answer)

        qc_match = apply_swaps_and_get_matching_circuit(qc_orig)
        file_problem = f"experiment_setup/circuits/zou_circuits/problems_ring_27/parallel_circuit_{num_qubits}_{num_layers}.qasm"
        dump(qc_match, file_problem)

        qc_list.append((qc_orig, qc_match))

        print("Done with num_layers = ", num_layers)
    return qc_list

In [6]:
num_qubits = 27
num_layers_list = range(1, 101)
cm = CouplingMap.from_ring(num_qubits)


In [7]:
qc_list = create_parallel_circuits_set(num_qubits, num_layers_list, cm)

Done with num_layers =  1
Done with num_layers =  2
Done with num_layers =  3
Done with num_layers =  4
Done with num_layers =  5
Done with num_layers =  6
Done with num_layers =  7
Done with num_layers =  8
Done with num_layers =  9
Done with num_layers =  10
Done with num_layers =  11
Done with num_layers =  12
Done with num_layers =  13
Done with num_layers =  14
Done with num_layers =  15
Done with num_layers =  16
Done with num_layers =  17
Done with num_layers =  18
Done with num_layers =  19
Done with num_layers =  20
Done with num_layers =  21
Done with num_layers =  22
Done with num_layers =  23
Done with num_layers =  24
Done with num_layers =  25
Done with num_layers =  26
Done with num_layers =  27
Done with num_layers =  28
Done with num_layers =  29
Done with num_layers =  30
Done with num_layers =  31
Done with num_layers =  32
Done with num_layers =  33
Done with num_layers =  34
Done with num_layers =  35
Done with num_layers =  36
Done with num_layers =  37
Done with 